In [17]:
from datetime import date, timedelta
import os
import dill as pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer, utils
from abautomator import config, describer, analyzer

from bokeh.io import output_notebook
from bokeh.plotting import show, save
# from bokeh.models import MultiChoice, CustomJS, Button
from bokeh.layouts import column, row

output_notebook()

Loading BokehJS ...

In [18]:
# %pip install -U pandas

In [19]:
EXP_NAME = "ProtedtUpSellExp20224023_"
CTRL_NAME = 'ctrl'

In [24]:
def filter_coll_metrics(coll, column, value):
    for metric in coll.metrics:
        metric.user_metric_df = metric.user_metric_df[metric.user_metric_df[column] == value]
    
    return coll

In [26]:
local_coll = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{EXP_NAME}_coll.p"), "rb" )
)

In [43]:
import copy

for value in ["all", "android", "ios"]:
    curr_coll = copy.deepcopy(local_coll)
    if value != "all":
        curr_coll = filter_coll_metrics(curr_coll, column="device_type", value=value)

    print(f"describing data for {value}")
    desc = describer.Describer(
        metrics=curr_coll.metrics
    )
    outcomes_dict = desc.describe_data(exp_name=EXP_NAME)
    
    print(f"analyzing data for {value}")
    analy =  analyzer.Analyzer(
        outcomes=outcomes_dict,
        ctrl_name=CTRL_NAME,
    )

    pickle.dump(
        analy, open(f"{EXP_NAME}_analy_{value}.p", "wb" )
    )

describing data for all
analyzing data for all
describing data for android
analyzing data for android
describing data for ios
analyzing data for ios


In [14]:
# analy.base_df.head()

In [29]:
raw_df = analy.get_rel_diff_confidence_intervals()
pd.set_option('display.max_colwidth', None)
# raw_df

In [30]:
# list(raw_df["metric"])
metric_list = list(raw_df["metric"])

In [37]:
cond_order = ["tx_w_protect_upsell"]

primary_metric_list = ['n_feed_views',
#  'pct_feed_views',
 'n_feed_taps',
#  'pct_feed_taps',
#  'n_all_feed_shares',
 'pct_all_feed_shares',
#  'n_direct_feed_shares',
#  'pct_direct_feed_shares',
#  'n_indirect_feed_shares',
#  'pct_indirect_feed_shares',
#  'n_signup_activation',
 'pct_signup_activation',
]
# 
secondary_metric_list = [
 'n_all_sessions',
 'pct_all_sessions',
 'n_organic_sessions',
 'pct_organic_sessions',
 'n_push_driven_sessions',
 'pct_push_driven_sessions',
 'n_incident_views',
 'pct_incident_views',
 'n_incident_share_attempts',
 'pct_incident_share_attempts',
]

guardrail_metric_list = [
 'n_chats',
 'pct_chats',
 'n_friend_invites',
 'pct_friend_invites',
 'n_trial_starts',
 'pct_trial_starts',
]

In [38]:
output_df = raw_df.sort_values(by=['abs_mean'], ascending=False)#[["metric", "abs_mean"]]
# output_df

In [52]:
from bokeh.models import Title
fig.add_layout(Title(text="Bottom Centered Title", align="center"), "above")
show(fig)

In [48]:
df = utils.order_categories(output_df, primary_metric_list, cond_order)
df = utils.remove_categories_w_no_order(df)
source = utils.convert_df_to_source(df)

vis = RelDiffVisualizer(source)
fig = vis.get_figure()
# to_show = vis.get_layout()
# show(to_show)

NaN values found:                exp_cond                       metric   tx_mean     tx_std  \
12  tx_w_protect_upsell               n_all_sessions  6.545272   8.908460   
18  tx_w_protect_upsell             n_incident_views  7.156453  14.489921   
14  tx_w_protect_upsell           n_organic_sessions  3.167038   5.875472   
16  tx_w_protect_upsell       n_push_driven_sessions  3.378234   5.164780   
15  tx_w_protect_upsell         pct_organic_sessions  0.801497   0.398875   
13  tx_w_protect_upsell             pct_all_sessions  0.955604   0.205974   
8   tx_w_protect_upsell       n_indirect_feed_shares  0.039611   0.733325   
4   tx_w_protect_upsell            n_all_feed_shares  0.048943   0.758502   
9   tx_w_protect_upsell     pct_indirect_feed_shares  0.019140   0.137017   
1   tx_w_protect_upsell               pct_feed_views  0.998036   0.044275   
23  tx_w_protect_upsell                    pct_chats  0.033771   0.180641   
21  tx_w_protect_upsell  pct_incident_share_attempts  0.09

In [47]:
to_show.__dict__

{'_id': '1282',
 '_document': None,
 '_temp_document': None,
 '_event_callbacks': {},
 '_callbacks': {},
 '_property_values': {'children': [Figure(id='1200', ...),
   Column(id='1281', ...)]},
 '_unstable_default_values': {'subscribed_events': [],
  'css_classes': [],
  'js_event_callbacks': {},
  'margin': (0, 0, 0, 0),
  'tags': [],
  'js_property_callbacks': {}},
 '_unstable_themed_values': {},
 '_initialized': True}

In [34]:
save(to_show, "/home/jovyan/work/notebooks/android-primary-metrics.html")

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/jovyan/work/notebooks/android-primary-metrics.html'

In [35]:
df = utils.order_categories(output_df, secondary_metric_list, cond_order)
df = utils.remove_categories_w_no_order(df)
source = utils.convert_df_to_source(df)

vis = RelDiffVisualizer(source)
to_show = vis.get_layout()
show(to_show)

In [36]:
save(to_show, "/home/jovyan/work/notebooks/android-secondary-metrics.html")

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/jovyan/work/notebooks/android-secondary-metrics.html'

In [37]:
df = utils.order_categories(output_df, guardrail_metric_list, cond_order)
df = utils.remove_categories_w_no_order(df)
source = utils.convert_df_to_source(df)

vis = RelDiffVisualizer(source)
to_show = vis.get_layout()
show(to_show)

In [38]:
save(to_show, "/home/jovyan/work/notebooks/android-guardrail-metrics.html")

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/jovyan/work/notebooks/android-guardrail-metrics.html'